# Tutorial 4 : The lattice-Boltzmann method in ESPResSo - Part 3

### 5.2 Step 2: Diffusion of a polymer

One of the typical applications of **ESPResSo** is the simulation of polymer chains with a bead-spring-model. For this we need a repulsive interaction between all beads, for which one usually takes a shifted and truncated Lennard-Jones (so-called Weeks–Chandler–Andersen or WCA) interaction, and additionally a bonded interaction between adjacent beads to hold the polymer together. You have already learned that the command

```python
system.non_bonded_inter[0, 0].lennard_jones.set_params(
    epsilon=1.0, sigma=1.0, shift=0.25, cutoff=1.1225)
```

creates a Lennard-Jones interaction with $\varepsilon=1.$, $\sigma=1.$,
$r_\text{cut} = 1.1225$ and $\varepsilon_\text{shift}=0.25$ between particles
of type 0, which is the desired repulsive interaction. The command

```python
fene = espressomd.interactions.FeneBond(k=7, r_0=1, d_r_max=2)
```

creates a <tt>FeneBond</tt> object (see **ESPResSo** manual for the details). What is left to be done is to add this bonded interaction to the system via

```python
system.bonded_inter.add(fene)
```

and to apply the bonded interaction to all monomer pairs of the polymer as shown in the script below.

**ESPResSo** provides a function that tries to find monomer positions that minimize the overlap between
monomers of a chain, *e.g.*:

```python
positions = espressomd.polymer.linear_polymer_positions(n_polymers=1,
                                                        beads_per_chain=10,
                                                        bond_length=1, seed=42,
                                                        min_distance=0.9)
```

which would create positions for a single polymer with 10 monomers. Please check the documentation for a more detailed description.

Furthermore we want to compute the diffusion constant of the polymer for different numbers of monomers.
For this purpose we can again use the multiple tau correlator. The following script computes the mean
squared displacement for the center of mass of the polymer as well as the average hydrodynamic radius
$R_h$, end-to-end distance $R_F$ and radius of gyration $R_g$.

How do $R_h$, $R_g$, $R_F$ and the diffusion coefficient $D$ depend on the number of monomers?
You can refer to the Flory theory of polymers, and assume you are simulating a real polymer in a
good solvent, with Flory exponent $\nu \approx 0.588$.

In [ ]:
import logging
import sys

import numpy as np

import espressomd
import espressomd.accumulators
import espressomd.observables
import espressomd.polymer

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

espressomd.assert_features(['LENNARD_JONES'])

# Setup constants
TIME_STEP = 0.01
LOOPS = 4000
STEPS = 100

# System setup
system = espressomd.System(box_l=[12.0, 12.0, 12.0])
system.cell_system.skin = 0.4

# Lennard-Jones interaction
system.non_bonded_inter[0, 0].lennard_jones.set_params(
    epsilon=1.0, sigma=1.0, shift="auto", cutoff=2.0**(1.0 / 6.0))

# Fene interaction
fene = espressomd.interactions.FeneBond(k=7, r_0=1, d_r_max=2)
system.bonded_inter.add(fene)

N_MONOMERS = np.array([6, 8, 10])

tau_results = []
msd_results = []
rh_results = []
re_results = []
rg_results = []
for index, N in enumerate(N_MONOMERS):
    logging.info("Polymer size: {}".format(N))
    # create a linear polymer with Fene bonds
    positions = espressomd.polymer.linear_polymer_positions(n_polymers=1,
                                                            beads_per_chain=N,
                                                            bond_length=1, seed=42,
                                                            min_distance=0.9)
    for i, pos in enumerate(positions[0]):
        pid = len(system.part)
        system.part.add(id=pid, pos=pos)
        if i > 0:
            system.part[pid].add_bond((fene, pid - 1))

    logging.info("Warming up the polymer chain.")
    system.time_step = 0.002
    system.integrator.set_steepest_descent(
        f_max=1.0,
        gamma=10,
        max_displacement=0.01)
    system.integrator.run(2000)
    system.integrator.set_vv()
    logging.info("Warmup finished.")

    logging.info("Equilibration.")
    system.time_step = TIME_STEP
    system.thermostat.set_langevin(kT=1.0, gamma=50, seed=42)
    system.integrator.run(2000)
    logging.info("Equilibration finished.")

    system.thermostat.turn_off()

    lbf = espressomd.lb.LBFluidGPU(
        kT=1,
        seed=42,
        agrid=1,
        dens=1,
        visc=5,
        tau=TIME_STEP)
    system.actors.add(lbf)
    system.thermostat.set_lb(LB_fluid=lbf, seed=42, gamma=5)

    logging.info("Warming up the system with LB fluid.")
    system.integrator.run(1000)
    logging.info("Warming up the system with LB fluid finished.")

    # configure correlator
    com_pos = espressomd.observables.ComPosition(ids=range(N))
    correlator = espressomd.accumulators.Correlator(
        obs1=com_pos, tau_lin=16, tau_max=LOOPS * STEPS, delta_N=1,
        corr_operation="square_distance_componentwise", compress1="discard1")
    system.auto_update_accumulators.add(correlator)

    logging.info("Sampling started.")
    rhs = np.zeros(LOOPS)
    res = np.zeros(LOOPS)
    rgs = np.zeros(LOOPS)
    for i in range(LOOPS):
        system.integrator.run(STEPS)
        rhs[i] = system.analysis.calc_rh(
            chain_start=0,
            number_of_chains=1,
            chain_length=N)[0]
        res[i] = system.analysis.calc_re(
            chain_start=0,
            number_of_chains=1,
            chain_length=N)[0]
        rgs[i] = system.analysis.calc_rg(
            chain_start=0,
            number_of_chains=1,
            chain_length=N)[0]

    logging.info("Sampling finished.")

    # store results
    correlator.finalize()
    corrdata = correlator.result()
    tau = correlator.lag_times()
    msd = np.sum(corrdata, axis=1)
    tau_results.append(tau)
    msd_results.append(msd)
    rh_results.append([np.average(rhs), np.std(rhs)])
    re_results.append([np.average(res), np.std(res)])
    rg_results.append([np.average(rgs), np.std(rgs)])

    # reset system
    system.part.clear()
    system.thermostat.turn_off()
    system.actors.clear()
    system.auto_update_accumulators.clear()

rh_results = np.array(rh_results)
re_results = np.array(re_results)
rg_results = np.array(rg_results)
tau_results = np.array(tau_results)
msd_results = np.reshape(msd_results, [len(N_MONOMERS),-1])

## References

[1] S. Succi. *The lattice Boltzmann equation for fluid dynamics and beyond.* Clarendon Press, Oxford, 2001.  
[2] B. Dünweg and A. J. C. Ladd. *Advanced Computer Simulation Approaches for Soft Matter Sciences III*, chapter II, pages 89–166. Springer, 2009.  
[3] B. Dünweg, U. Schiller, and A.J.C. Ladd. Statistical mechanics of the fluctuating lattice-Boltzmann equation. *Phys. Rev. E*, 76:36704, 2007.  
[4] P. G. de Gennes. *Scaling Concepts in Polymer Physics*. Cornell University Press, Ithaca, NY, 1979.  
[5] M. Doi. *Introduction to Polymer Physics.* Clarendon Press, Oxford, 1996.  
[6] Michael Rubinstein and Ralph H. Colby. *Polymer Physics.* Oxford University Press, Oxford, UK, 2003.  
[7] Daan Frenkel and Berend Smit. *Understanding Molecular Simulation.* Academic Press, San Diego, second edition, 2002.